In [1]:
import tensorflow as tf
import numpy as np

In [2]:
NORM = np.load('Heartbeats/Normal/N_beats.npy',allow_pickle = True)
ARRH = np.load('Heartbeats/Arrhythmia/A_beats.npy',allow_pickle = True)

print('loaded')
NORM = list(NORM)
ARRH = list(ARRH)
print('listed')

NORM = tf.ragged.constant(NORM)
ARRH = tf.ragged.constant(ARRH)
print('ragged')

loaded
listed
ragged


In [3]:
arrh_labels = np.array(tf.cast(np.array(np.ones(int(ARRH.shape[0]))),tf.float16))
norm_labels = np.array(tf.cast(np.array(np.zeros(int(NORM.shape[0]))),tf.float16))

print(norm_labels.shape,arrh_labels.shape,norm_labels[0],arrh_labels[0],type(norm_labels.astype(dtype = 'int32')),type(arrh_labels))

(130665,) (163323,) 0.0 1.0 <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [4]:
def cast16(beat,label):
    beat = tf.cast(beat,tf.float16)
    return beat,label

def one_hot(beat, d_model):
    if type(d_model) != int:
        raise TypeError("d_model must be an int type")
    encodings = []
    beat = tf.cast((tf.cast(beat,tf.float16)*int(d_model)),tf.int64)
    
    # print(beat.dtype) # troubleshooting
    
    for i in beat:
#         i = tf.round(i)
        i = tf.cast(i,tf.int32)
        # print(i,i.dtype)
        if i == 0:
            encodings.append(list(np.zeros(d_model)))
        else:
            encodings.append(list(np.zeros(i-1)) + list(np.ones(1)) + list(np.zeros(d_model-i)))
    return encodings

def input_embeddings(encodings,max_sequence_length,d_model):
    
    temp = int(max_sequence_length) - int(len(list(encodings)))
    for i in range(temp):
        encodings.append(list(np.zeros(int(d_model))))
    return encodings

In [5]:
all_data = tf.concat([NORM,ARRH],axis = 0)
all_labels = tf.concat([norm_labels,arrh_labels],axis = 0)

print(all_data.shape,all_labels.shape)
print(tf.cast(all_data,tf.float16).dtype,all_labels.dtype)
# print(type(all_data.to_list()),type([all_labels]))

(293988, None) (293988,)
<dtype: 'float16'> <dtype: 'float16'>


In [18]:
print(np.array(tf.cast(input_embeddings(one_hot(all_data[0],512),875,512),dtype='float16')).astype(dtype = np.int8).dtype)
print([x for x in np.array(tf.cast(input_embeddings(one_hot(all_data[0],512),875,512),dtype='float16')).astype(dtype = np.int4)])

int8


AttributeError: module 'numpy' has no attribute 'int4'

In [7]:
# data = all_data.to_list()
# labels = [all_labels]

# print(type(data),type(labels))

In [8]:
np.array(tf.cast(input_embeddings(one_hot(all_data[0],512),875,512),dtype='float16')).max()

1.0

In [9]:
# data = []
# labels = []

# for i in all_data:

#     temp = np.array(tf.cast(input_embeddings(one_hot(i,512),875,512),dtype='float16'))
#     data.append(temp)

# print('data done')

# labels = np.array(all_labels).tolist()


In [10]:
def positional_encoding(length, depth):
#     depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (500**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

#     pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)],axis=-1) 
    pos_encoding = np.sin(angle_rads)

    return tf.cast(pos_encoding, dtype=tf.float16)